<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f6ffa4ffbe96fac1284cce025e3fda066470fdd2414da971b7e6a3d5087330dd
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-28 10:13:52
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -16.04 K (-0.11%)
Current PnL: -22.55 L (-14.78%)
CY Booked + Current PnL: -8.66 L (-5.68%)
-------------------
Total profit:  1.61 L
Total loss:  -24.16 L
-------------------
Total Booked + Current PnL: 18.32 L (14.58%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.7%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.96 L (61.41%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.44%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.0,0.68,-8.40,9.19,0.02,14695.0,-14661.0,159898.0,143.86,68.0,M-SC,10.47,234.0,-1.00,1.12,39.08,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.0,-1.03,-15.00,17.76,0.09,15213.0,-15116.0,85661.0,98.72,44.0,M-SC,3.58,253.0,-0.99,0.60,12.54,OX40N,NTT,DURABLES
83,VOLTAS,1530.0,-1.58,7.64,11.20,19.69,23115.0,14643.0,206385.0,-2.87,50.0,X-MC,3.32,78.0,0.63,1.44,14.95,XY25,NTT,AC
43,ITC,452.0,-0.28,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,44.0,X-LC,1.21,5.0,-0.13,1.38,3.98,X40,NTT,FMCG
52,MEDANTA,1486.0,-0.79,3.55,19.83,24.08,26067.0,4501.0,131451.0,-4.85,46.0,X-SC,5.89,89.0,0.17,0.92,23.86,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,3.66,-2.15,38.65,35.67,119447.0,-6795.0,309047.0,-13.74,63.0,X-LC,5.71,4.0,-0.06,2.16,11.40,X40N,ATH,FMCG
29,HAPPSTMNDS,1480.71,1.28,-37.13,194.03,84.85,156342.0,-47589.0,80576.0,-25.24,48.0,H-SC,14.20,147.0,-0.30,0.56,3.41,AR,ATH,IT
84,WHIRLPOOL,2270.00,1.17,-13.75,110.93,81.92,109832.0,-15788.0,99010.0,-49.48,26.0,M-SC,2.89,236.0,-0.14,0.69,17.21,XR,NTT,DURABLES
33,HINDUNILVR,2922.00,1.09,-4.63,19.18,13.67,47024.0,-11893.0,245170.0,-11.77,51.0,X-LC,3.97,14.0,-0.25,1.71,15.74,XY25,NTT,FMCG
46,JSWINFRA,342.00,1.06,-8.12,25.41,15.23,46773.0,-16275.0,184072.0,-21.45,34.0,X-MC,6.95,66.0,-0.35,1.29,21.51,X40N,NTT,REALTY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RAJOOENG,143.10,-2.27,-26.53,90.01,39.61,67787.0,-27190.0,75310.0,-50.27,37.0,H-SC,21.09,135.0,-0.40,0.53,6.15,AR,ATH,MISC
83,VOLTAS,1530.00,-1.58,7.64,11.20,19.69,23115.0,14643.0,206385.0,-2.87,50.0,X-MC,3.32,78.0,0.63,1.44,14.95,XY25,NTT,AC
67,SIS,528.00,-1.48,-21.99,55.45,21.26,48026.0,-24421.0,86611.0,2046.11,55.0,H-SC,3.74,166.0,-0.51,0.60,17.02,OX40N,NTT,MISC
48,KPIGREEN,730.66,-1.42,-12.99,68.90,46.95,75111.0,-16282.0,109015.0,-19.60,28.0,H-SC,2.28,154.0,-0.22,0.76,36.57,MH,ATH,POWER
47,KANSAINER,340.00,-1.26,-21.99,45.45,13.47,95615.0,-59292.0,210375.0,-67.92,35.0,H-SC,1.63,158.0,-0.62,1.47,6.97,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.4,0.22,106.26,106.72,131350.0,276.0,123612.0,-50.44,39.0,H-SC,2.78,139.0,0.0,0.86,23.65,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.30,-4.83,118.47,107.91,166465.0,-7132.0,140512.0,-22.58,36.0,M-SC,10.66,216.0,-0.04,0.98,0.47,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.68,-8.40,9.19,0.02,14695.0,-14661.0,159898.0,143.86,68.0,M-SC,10.47,234.0,-1.00,1.12,39.08,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-1.03,-15.00,17.76,0.09,15213.0,-15116.0,85661.0,98.72,44.0,M-SC,3.58,253.0,-0.99,0.60,12.54,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.26,-21.99,45.45,13.47,95615.0,-59292.0,210375.0,-67.92,35.0,H-SC,1.63,158.0,-0.62,1.47,6.97,XY24,NTT,PAINTS
67,SIS,528.00,-1.48,-21.99,55.45,21.26,48026.0,-24421.0,86611.0,2046.11,55.0,H-SC,3.74,166.0,-0.51,0.60,17.02,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.25,2.36,68.30,72.27,118462.0,3996.0,173444.0,-11.98,59.0,M-LC,3.87,99.0,0.03,1.21,8.68,XR,NTT,IT
38,INDIAMART,4810.62,0.40,0.22,106.26,106.72,131350.0,276.0,123612.0,-50.44,39.0,H-SC,2.78,139.0,0.00,0.86,23.65,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.25,2.36,68.30,72.27,118462.0,3996.0,173444.0,-11.98,59.0,M-LC,3.87,99.0,0.03,1.21,8.68,XR,NTT,IT
38,INDIAMART,4810.62,0.40,0.22,106.26,106.72,131350.0,276.0,123612.0,-50.44,39.0,H-SC,2.78,139.0,0.00,0.86,23.65,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.05,-1.12,37.16,35.63,77007.0,-2342.0,207232.0,-14.97,42.0,H-MC,3.76,119.0,-0.03,1.45,15.37,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.30,-4.83,118.47,107.91,166465.0,-7132.0,140512.0,-22.58,36.0,M-SC,10.66,216.0,-0.04,0.98,0.47,OX40N,ATH,CABLES
1,ABB,7934.00,0.11,-3.74,51.22,45.57,128988.0,-9787.0,251832.0,-38.17,60.0,H-MC,5.71,120.0,-0.08,1.76,7.16,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.06,-35.53,109.70,35.20,91005.0,-45712.0,82958.0,8.74,24.0,X-SC,13.35,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,600.00,-0.46,-15.60,68.47,42.18,159725.0,-43132.0,233278.0,-24.30,27.0,X-LC,1.95,3.0,-0.27,1.63,1.05,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.00,-13.83,72.28,48.45,124881.0,-27732.0,172774.0,-20.09,32.0,X-MC,7.53,64.0,-0.22,1.21,0.37,X40N,ATH,FINANCE
46,JSWINFRA,342.00,1.06,-8.12,25.41,15.23,46773.0,-16275.0,184072.0,-21.45,34.0,X-MC,6.95,66.0,-0.35,1.29,21.51,X40N,NTT,REALTY
22,DIXON,18931.72,-0.51,-6.44,29.95,21.58,39271.0,-9018.0,131121.0,-52.80,35.0,X-MC,6.42,56.0,-0.23,0.92,16.08,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.28,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,44.0,X-LC,1.21,5.0,-0.13,1.38,3.98,X40,NTT,FMCG
3,ACC,3906.0,-0.12,-21.50,109.11,64.15,203807.0,-51161.0,186790.0,-54.13,53.0,X-SC,1.53,82.0,-0.25,1.30,4.78,XY24,BTT,CEMENT
15,CAMS,4762.0,-0.48,1.56,22.87,24.79,58499.0,3934.0,255790.0,-3.25,48.0,X-SC,1.87,86.0,0.07,1.79,25.06,X40N,NTT,MISC
66,SIEMENS,4671.5,-0.30,-11.28,41.47,25.51,68467.0,-20995.0,165100.0,1.35,64.0,H-LC,1.88,49.0,-0.31,1.15,21.67,AR,ATH,ELECTRICAL
76,TMPV,600.0,-0.46,-15.60,68.47,42.18,159725.0,-43132.0,233278.0,-24.30,27.0,X-LC,1.95,3.0,-0.27,1.63,1.05,XY24,NTT,AUTO


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.06,-35.53,109.70,35.20,91005.0,-45712.0,82958.0,8.74,24.0,X-SC,13.35,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.07,-0.81,-5.67,33.64,26.07,51960.0,-9280.0,154460.0,-30.55,36.0,X-MC,9.55,55.0,-0.18,1.08,0.21,X40,ATH,APPARELS
9,BAJAJHFL,181.50,0.00,-13.83,72.28,48.45,124881.0,-27732.0,172774.0,-20.09,32.0,X-MC,7.53,64.0,-0.22,1.21,0.37,X40N,ATH,FINANCE
55,PGHH,17907.65,0.39,-4.17,39.58,33.76,76172.0,-8370.0,192450.0,-32.38,36.0,X-MC,3.86,57.0,-0.11,1.34,0.81,X40,ATH,FMCG
76,TMPV,600.00,-0.46,-15.60,68.47,42.18,159725.0,-43132.0,233278.0,-24.30,27.0,X-LC,1.95,3.0,-0.27,1.63,1.05,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.07,-28.07,98.58,42.84,46097.0,-18245.0,46761.0,-53.17,41.0,X-SC,37.59,83.0,-0.40,0.33,1.75,XY24,NTT,MISC
60,RELAXO,1176.00,0.22,-46.76,186.10,52.31,143812.0,-67883.0,77277.0,-43.28,39.0,X-SC,4.50,91.0,-0.47,0.54,3.44,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.06,-35.53,109.70,35.20,91005.0,-45712.0,82958.0,8.74,24.0,X-SC,13.35,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.51,-6.44,29.95,21.58,39271.0,-9018.0,131121.0,-52.80,35.0,X-MC,6.42,56.0,-0.23,0.92,16.08,X40N,ATH,IT
52,MEDANTA,1486.00,-0.79,3.55,19.83,24.08,26067.0,4501.0,131451.0,-4.85,46.0,X-SC,5.89,89.0,0.17,0.92,23.86,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.96,0.21,-14.60,39.67,19.28,117205.0,-50505.0,295451.0,-25.57,59.0,X-LC,7.11,1.0,-0.43,2.06,9.22,X40,ATH,IT
41,INFY,2275.00,-0.15,6.47,45.46,54.87,151442.0,20244.0,333132.0,-17.09,62.0,X-LC,2.47,2.0,0.13,2.33,15.29,X40,BTT,IT
76,TMPV,600.00,-0.46,-15.60,68.47,42.18,159725.0,-43132.0,233278.0,-24.30,27.0,X-LC,1.95,3.0,-0.27,1.63,1.05,XY24,NTT,AUTO
82,VBL,671.64,3.66,-2.15,38.65,35.67,119447.0,-6795.0,309047.0,-13.74,63.0,X-LC,5.71,4.0,-0.06,2.16,11.40,X40N,ATH,FMCG
43,ITC,452.00,-0.28,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,44.0,X-LC,1.21,5.0,-0.13,1.38,3.98,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.79,-36.70,109.92,32.89,53723.0,-28332.0,48875.0,-703.29,65.0,L-MC,5.65,259.0,-0.53,0.34,34.60,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.18,-12.19,106.89,81.67,88054.0,-11432.0,82378.0,7405.56,54.0,L-SC,19.60,271.0,-0.13,0.58,59.53,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.20,-11.69,58.01,39.54,51825.0,-11821.0,89338.0,-30.90,68.0,M-SC,6.68,220.0,-0.23,0.62,25.97,AR,ATH,AUTO
51,MASFIN,398.61,0.30,-6.52,30.56,22.05,27990.0,-6390.0,91590.0,-19.60,46.0,H-SC,6.93,164.0,-0.23,0.64,33.13,XR,ATH,FINANCE
70,SURYODAY,216.00,-0.38,-20.20,54.18,23.03,77424.0,-36169.0,142902.0,55.70,47.0,H-SC,9.56,167.0,-0.47,1.00,41.79,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.68,-8.40,9.19,0.02,14695.0,-14661.0,159898.0,143.86,68.0,M-SC,10.47,234.0,-1.00,1.12,39.08,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.21,0.42,20.44,20.95,51677.0,1056.0,252824.0,-0.63,74.0,X-LC,15.16,31.0,0.02,1.77,36.21,X40,ATH,PAINTS
50,LTIM,7201.88,0.47,10.68,18.96,31.66,51653.0,26279.0,272430.0,6.30,65.0,H-LC,13.55,43.0,0.51,1.90,51.16,X200,ATH,IT
79,UNITDSPR,1644.00,0.32,6.55,13.35,20.77,33304.0,15325.0,249469.0,-2.78,62.0,X-MC,3.78,62.0,0.46,1.74,13.92,X40N,NTT,BREWERIES
40,INDUSINDBK,1800.00,0.79,-36.70,109.92,32.89,53723.0,-28332.0,48875.0,-703.29,65.0,L-MC,5.65,259.0,-0.53,0.34,34.60,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.92
1,25,44.84
2,50,75.68


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.44
MC    29.21
LC    25.37
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.80
X40      21.21
X40N     12.04
XR        9.52
AR        9.10
XY25      9.09
OX40N     7.62
X200      1.90
SR        0.98
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.85
X-MC    22.72
X-LC    20.14
M-SC    11.92
X-SC     8.26
H-MC     4.77
H-LC     3.05
L-SC     1.41
M-MC     1.38
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.90,-4.00,37.63
IT,13.82,-14.26,73.25
FINANCE,9.47,-14.25,62.53
MISC,7.39,-22.90,73.25
ELECTRICAL,6.00,-9.27,49.30
PAINTS,5.96,-8.25,24.33
INSURANCE,4.40,-0.27,35.60
PHARMA,3.87,-0.90,33.19
AUTO,3.37,-19.04,70.08


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3055492.0,21
XR,1299049.0,13
AR,1268850.0,10
X40,1001898.0,14
X40N,726227.0,9
OX40N,699446.0,10
XY25,338622.0,6
SR,280100.0,2
MH,75111.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3562593.0,25
M-SC,1351693.0,15
X-MC,1230768.0,16
X-LC,879407.0,11
X-SC,737523.0,8
H-MC,395639.0,3
L-SC,259595.0,3
H-LC,120120.0,2
M-LC,118462.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1216409.0      6
           AR         873718.0      5
           XR         796128.0      7
M-SC       XY24       781563.0      6
X-MC       X40        444564.0      7
X-LC       X40        389965.0      5
X-MC       XY24       363418.0      3
H-SC       OX40N      321127.0      4
X-SC       X40N       294183.0      3
M-SC       OX40N      290299.0      5
H-SC       SR         280100.0      2
X-SC       XY24       275971.0      3
X-MC       X40N       244229.0      4
X-LC       XY24       228487.0      2
H-MC       AR         205995.0      2
           XY24       189644.0      1
X-LC       X40N       187815.0      2
X-MC       XY25       178557.0      2
L-SC       XR         171575.0      2
X-SC       X40        167369.0      2
M-SC       XR         159161.0      2
           AR         120670.0      2
M-LC       XR         118462.0      1
L-SC       OX40N       88020.0      1
H-SC       MH          75111.0      1
X-LC       XY25        73140.0      2
H-LC       AR          68467.0      1
L-MC       XR          53723.0      1
H-LC       X200        51653.0      1
M-MC       XY25        47031.0      1
L-LC       XY25        39894.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
